In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Scrape all articles in special index

In [2]:
url = "https://kbp.wikipedia.org/wiki/Sp%C3%A9cial:Index?prefix=&namespace=0" #Starting url
nextLink = True
titlesList = []

#### Get content of list of articles

In [3]:
while(nextLink):
    #Send request
    response = requests.get(url)
    html = response.text
    pageSoup = BeautifulSoup(html,"html5lib")
    #Get list of article titles
    articles = pageSoup.findAll('a', {'class' : 'mw-redirect'})
    for article in articles:
        titlesList.append(article.text)
    #Check if there is a next link
    links = pageSoup.findAll('div', {'class' : 'mw-prefixindex-nav'})
    if(links): 
        nextLink = True
        url = 'https://kbp.wikipedia.org/' + links[0].find('a')['href']
    else :
        nextLink = False

In [4]:
print(titlesList)

['"Albert Camus"', '"Albert Einstein"', '"Alexandre Sɔsɔ"', '"Alizerii you tɔm"', '"Charles Darwin"', '"Claude Ptolémée"', '"Galilée"', '"Heelim"', '"Kazayaɣ"', '"Keziye"', '"Kilimanjaroo pʋʋ"', '"Krɩstʋ...ŋgbɛyɛ"', '"Kɔfɩ"', '"Mɩndʋ"', '"Niger pɔɔ"', '"Papa François"', '"Pɩsatʋ"', '"Requin"', '"Taataa"', '"Tu"', '"Tuduwa"', '"Tɩgbayʋ"', '"Wɩsɩ kʋdɔŋ"', '"Ɛsɔdaa lɔɔɖɩyɛ"', '.Ɛndɩyɔm', 'Abasidi (Abbasside)', 'Abuuja (Abuja)', 'Abû Nuwâsest', 'Abɩɩjaŋ (Abidjan)', 'Adisi-Abeba (Addis-Abeba)', 'Afganistani (Afghanistan)', 'Afrika Hadɛ kiŋ', 'Afrika Hadɛ kiŋ (Afrique du Sud)', 'Afrika Wɩsɩɖʋɖʋyɛ Ajɛyɛ Tɔsʋʋŋgbɛyɛ (AWATƆŊ)/(CEDEAO)', 'Afrika nɛ wɩsɩ ɖɩɖʋyɛ taa lɩŋgamɩŋ', 'Afrika wɩsɩ ɖɩɖʊyɛ taa lɩŋgamɩŋ', 'Afrɩka tɛtʋ wandamm(Continent africain)', 'Afrɩka tɛtʋ wandamm (Continent africain)', 'Afʊwanɩ (Avoine)', 'Ajɛyɛ ñʊŋ yɔɔ caɣʊ tɔm ɖɩzʊʊ (La décolonisation)', 'Akaŋ', 'Akra (Accra)', 'Akropɔɔlɩ (Acropole)', 'Al-Khawarizmi', 'Alexandre Nikolaïevitch Scriabine', 'Alexandre Sergueïevitch Pouch

In [5]:
import wikipedia as wk

In [6]:
#Simple test to check if the api is working
print(wk.summary("ewe"))

"Ew!" is a song by American television host and comedian Jimmy Fallon, featuring American rapper will.i.am. The song is based on a sketch from The Tonight Show Starring Jimmy Fallon, in which Fallon and guests play teenage girls constantly disgusted by things around them. 
The song debuted on The Tonight Show on October 6, 2014, with Fallon urging viewers to get the song to chart on the Billboard charts. It debuted on the Billboard Hot 100 one week later at number 26.


In [7]:
wk.set_lang("kbp")

In [8]:
contentList = []
urlList = []
categoryList = []
i=0
for current in titlesList :
    print(i)
    i = i +1
    try:
        mypage = wk.WikipediaPage(title=current)
    except wikipedia.exceptions.PageError as e:
        continue
    if(mypage):
        #categories = mypage.categories
        #print(categories[0])
        #categoryList.append(categories[0])
        urlList.append(mypage.url)
        print(mypage.url)
        contentList.append(mypage.content)

0
https://kbp.wikipedia.org/wiki/Albert_Camus
1
https://kbp.wikipedia.org/wiki/Albert_Einstein


KeyboardInterrupt: 

In [87]:
contentless = [782,780,767,740,727,587,546,537,460,348,338,271,258,259]

In [110]:
n = len(titlesList)
finalTitles =  [ titlesList[index] for index in range(n) if index not in contentless ]
len(finalTitles)

827

In [114]:
df = pd.DataFrame(list(zip(finalTitles,contentList, urlList)), 
               columns =['title','Content', 'Url']) 

In [116]:
len(df)

827

In [118]:
df.to_csv(r'/home/godson/Documents/NLPProject/articles.csv')

### SCRAPE ARTICLES BY CATEGORY

In [9]:
#This wrapper of the original api is required to retrieve all articles of a specific category
import wikipediaapi

In [10]:
url = "https://kbp.wikipedia.org/wiki/Sp%C3%A9cial:Index/Category:" #All categories url
categoriesList = []
articlesList = []

#### Get the list of categories

In [11]:
#Send request
response = requests.get(url)
html = response.text
pageSoup = BeautifulSoup(html,"html5lib")
#Get list of category names
categories = pageSoup.findAll('ul', {'class' : 'mw-prefixindex-list'})[0].findAll('a')
#print(categories)
for category in categories:
    categoriesList.append(category.text)

In [12]:
print(len(categoriesList))
categoriesList = list(set(categoriesList)) #We remove duplicates from the list
print(len(categoriesList))

313
313


In [13]:
wiki_wiki = wikipediaapi.Wikipedia('kbp')

In [14]:
for category in categoriesList:
    cat = wiki_wiki.page("Category:"+category)
    for c in cat.categorymembers.values():
        articlesList.append(c.title)

In [16]:
print(len(articlesList))
articlesList = list(set(articlesList)) #We remove duplicates from the list
print(len(articlesList))

1669
1669


In [17]:
contentList = []
urlList = []
categoryList = []
i=0
for current in articlesList :
    print(i)
    i = i +1
    try:
        mypage = wk.WikipediaPage(title=current)
    except wikipedia.exceptions.PageError as e:
        categoryList.append("empty")
        urlList.append("empty")
        contentList.append("empty")
        continue
    if(mypage):
        categories = mypage.categories
        print(categories)
        categoryList.append(categories)
        urlList.append(mypage.url)
        print(mypage.url)
        contentList.append(mypage.content)

0
['Catégorie:!Main category', 'Catégorie:Kʋdʋmɩŋ']
https://kbp.wikipedia.org/wiki/W%C9%A9s%C9%A9_k%CA%8Bd%C9%94%C5%8B
1
['Catégorie:Fiziiki', 'Catégorie:Kabɩyɛ takayɩhatʋ']
https://kbp.wikipedia.org/wiki/Nima_y%C9%94%C9%94_l%C9%A9ma%C9%A3z%C9%A9oy%C9%9B
2
['Catégorie:Kpɛli pɩyʋʋ pɩlʋʋ kpata']
https://kbp.wikipedia.org/wiki/B%C9%94r%C9%A9
3
['Catégorie:!Main category']
https://kbp.wikipedia.org/wiki/Avicenna
4
['Catégorie:Aazii', 'Catégorie:Ajɛya', 'Catégorie:Papuwasii Kinee Kifalʋʋ']
https://kbp.wikipedia.org/wiki/Papuwazii_Kinee_K%C9%A9fal%CA%8A%CA%8A
5
['Catégorie:!Main category', 'Catégorie:Ajɛya', 'Catégorie:Eerɔpʋ', 'Catégorie:Pɔritigaalɩ']
https://kbp.wikipedia.org/wiki/P%C9%94ritigaal%C9%A9
6
['Catégorie:!Main category', 'Catégorie:Minziiki', 'Catégorie:Minziiki wondu']
https://kbp.wikipedia.org/wiki/Sak%C9%A9sofon%C9%A9
7
['Catégorie:!Main category', 'Catégorie:Abalaa sɔsaa', 'Catégorie:Aŋgilitɛɛrɩ', 'Catégorie:Takayɩsɩ mayaa', 'Catégorie:Ɛyaa sɔsaa']
https://kbp.wikipedia.org

70
['Catégorie:Kʋnʋmɩŋ']
https://kbp.wikipedia.org/wiki/H%C9%9Bbra_k%CA%8An%CA%8A%C5%8B
71
['Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Politiki']
https://kbp.wikipedia.org/wiki/Kewiya%C9%A3_%C9%9Bja%C9%96%C9%9B
72
['Catégorie:!Main category']
https://kbp.wikipedia.org/wiki/Yo%C5%8B_loluu_n%C9%9B_%C9%A9-%C9%96%C9%A9z%CA%8B%CA%8B
73
['Catégorie:User it']
https://kbp.wikipedia.org/wiki/Cat%C3%A9gorie:User_it-1
74
['Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Tɛtɛ wondu cikpetu', 'Catégorie:Wezuu lɛɣtʋ']
https://kbp.wikipedia.org/wiki/P%C9%A9c%C9%A9ka
75
['Catégorie:Fiziiki', 'Catégorie:Kabɩyɛ takayɩhatʋ']
https://kbp.wikipedia.org/wiki/Way%C9%A9_y%C9%94%C9%94_h%C9%94%C9%94l%CA%8B%CA%8B_(la_surface)
76
['Catégorie:!Main category', 'Catégorie:Ɛyʋ pilinzi']
https://kbp.wikipedia.org/wiki/N%C9%A9%C5%8Bgba%C5%8Ba%C9%A3
77
['Catégorie:Etaazuunii', 'Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Sukulinaa kɩtɛzaa']
https://kbp.wikipedia.org/wiki/Yale_sukuli_k%C9%A9t%C9%9Bz%CA%8A%CA%8A
78
['Catégorie:!Main

134
['Catégorie:!Main category', 'Catégorie:Ajɛya', 'Catégorie:Amerika', 'Catégorie:Ameriki nɛ hadɛ kiŋ wandamm', 'Catégorie:Silii']
https://kbp.wikipedia.org/wiki/Silii
135
['Catégorie:!Main category', 'Catégorie:Paɣtʋ lɛɣtʋ']
https://kbp.wikipedia.org/wiki/Pa%C9%A3t%CA%8A_%C9%96%C9%94n%C9%9B
136
['Catégorie:Pɩnzɩ', 'Catégorie:Pɩnzɩ mɩnɩŋ nɛɛlɛ']
https://kbp.wikipedia.org/wiki/1979
137
['Catégorie:Filozofii']
https://kbp.wikipedia.org/wiki/L%C9%9Bl%CA%8A
138
['Catégorie:!Main category', 'Catégorie:Afrɩka', 'Catégorie:Madakasɩkarɩ', 'Catégorie:Tɛtʋ sɔsɔtʋ']
https://kbp.wikipedia.org/wiki/Antananarivo
139
['Catégorie:!Main category', 'Catégorie:Afrɩka', 'Catégorie:Kinee-Pɩsaʊ', 'Catégorie:Tɛtʋ sɔsɔtʋ']
https://kbp.wikipedia.org/wiki/Pisawo
140
['Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Samaɣ']
https://kbp.wikipedia.org/wiki/%C3%91%C9%A9m
141
['Catégorie:Afrɩka', 'Catégorie:Etiyoopii', 'Catégorie:Tɛtʋ sɔsɔtʋ']
https://kbp.wikipedia.org/wiki/Adisi-Abeba
142
['Catégorie:Afrɩka', 'Catégorie

201
['Catégorie:!Main category', 'Catégorie:Afrɩka', 'Catégorie:Ajɛya', 'Catégorie:Aŋgoola']
https://kbp.wikipedia.org/wiki/A%C5%8Bgoola
202
['Catégorie:!Main category', 'Catégorie:Filozofii']
https://kbp.wikipedia.org/wiki/Ciyam
203
['Catégorie:!Main category', 'Catégorie:Abalaa sɔsaa', 'Catégorie:Krɩstiyaniism', 'Catégorie:Ɛsɔ sɛʋ aŋgba yɔɔ cɔnɩyaa', 'Catégorie:Ɛyaa sɔsaa']
https://kbp.wikipedia.org/wiki/Yeesu_Nazar%C9%9B%C9%9Bt%C9%A9_t%CA%8B
204
['Catégorie:!Main category', 'Catégorie:Politiki']
https://kbp.wikipedia.org/wiki/Kede%C5%8Bga_Kpeekpe_Y%C9%94%C9%94_%C6%90y%CA%8A_Wala_T%C9%94m
205
['Catégorie:Siimi', 'Catégorie:Siimi taŋgayɩ ñɩɩɩ waa', 'Catégorie:Ñɩɣtʋ']
https://kbp.wikipedia.org/wiki/Palad%C9%A9y%C9%94m
206
['Catégorie:!Main category', 'Catégorie:Aazii', 'Catégorie:Ajɛya', 'Catégorie:Eerɔpʋ', 'Catégorie:Tuurkii', 'Catégorie:Wɩsɩ ɖɩlɩyɛ hɛkʋ taa hɔlʋʋ wandamm']
https://kbp.wikipedia.org/wiki/Tuurkii
207
['Catégorie:Siimi', 'Catégorie:Siimi taŋgayɩ ñɩɩɩ waa', 'Catégorie:Ñɩ

263
['Catégorie:Fransɩɩ', 'Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Keesi maɖa']
https://kbp.wikipedia.org/wiki/Arthur_Rimbaud
264
['Catégorie:Siimi', 'Catégorie:Siimi taŋgayɩ ñɩɩɩ waa', 'Catégorie:Ñɩɣtʋ']
https://kbp.wikipedia.org/wiki/P%C9%A9s%C9%A9m%CA%8At%C9%A9
265
['Catégorie:Caama', 'Catégorie:Eerɔpʋ', 'Catégorie:Kʋnʋmɩŋ', 'Catégorie:Otirisi']
https://kbp.wikipedia.org/wiki/Caama_k%CA%8An%CA%8A%C5%8B
266
['Catégorie:!Main category', 'Catégorie:Kʋdʋmɩŋ']
https://kbp.wikipedia.org/wiki/%C6%90s%C9%94_k%CA%8Ad%C9%94%C5%8B_(Variole)
267
['Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Kabɩyɛ ɛsakuliye']
https://kbp.wikipedia.org/wiki/Egbele
268
['Catégorie:Abalaa sɔsaa', 'Catégorie:Caama', 'Catégorie:Minziiki laɖaa', 'Catégorie:Yendu lubiyaa', 'Catégorie:Ɛyaa sɔsaa']
https://kbp.wikipedia.org/wiki/Richard_Wagner
269
['Catégorie:!Main category', 'Catégorie:Afrɩka', 'Catégorie:Keniya', 'Catégorie:Tɛtʋ sɔsɔtʋ']
https://kbp.wikipedia.org/wiki/Nayiiroobii
270
['Catégorie:!Main category']
https:/

...

1357
['Catégorie:!Main category', 'Catégorie:Caanaʋ tɔm', 'Catégorie:Siini']
https://kbp.wikipedia.org/wiki/Mi%C5%8B_Kewiya%C9%A3_K%C9%A9%C9%96%C9%9Bza%C9%A3
1358
['Catégorie:!Main category', 'Catégorie:Sɩpɔɔrɩ']
https://kbp.wikipedia.org/wiki/C%C9%9B%C9%9Bc%C9%9B_sa%CA%8B_ale%C9%A3ya
1359
['Catégorie:!Main category', 'Catégorie:Lɛɣtʋ']
https://kbp.wikipedia.org/wiki/Cat%C3%A9gorie:T%C9%A9nz%C9%A9_l%C9%9B%C9%A3t%CA%8B
1360
['Catégorie:!Main category', 'Catégorie:Wilaŋ']
https://kbp.wikipedia.org/wiki/Wikipediya
1361
['Catégorie:!Main category', 'Catégorie:Tɛtʋ wandamm']
https://kbp.wikipedia.org/wiki/Antar%C9%A9tiki_t%C9%9Bt%CA%8B_wandamm
1362
['Catégorie:!Main category', 'Catégorie:Abalaa sɔsaa', 'Catégorie:Caama', 'Catégorie:Kewiyaɣ Kɩkpɛndaɣ', 'Catégorie:Minziiki laɖaa', 'Catégorie:Yendu lubiyaa', 'Catégorie:Ɛyaa sɔsaa']
https://kbp.wikipedia.org/wiki/Georg_Friedrich_Haendel
1363
['Catégorie:Pɩnzɩ', 'Catégorie:Pɩnzɩ mɩnɩŋ nɛɛlɛ']
https://kbp.wikipedia.org/wiki/1911
1364
['Catégorie:

1421
['Catégorie:Afrɩka', 'Catégorie:Togo', 'Catégorie:Togo taa tɛtʋ', 'Catégorie:Tɛtʋ sɔsɔtʋ']
https://kbp.wikipedia.org/wiki/%C5%8Aw%C9%9Bc%C9%A9
1422
['Catégorie:!Main category']
https://kbp.wikipedia.org/wiki/Ampiiri_paca%C9%A3_ma-%C3%B1%CA%8A%CA%8A_y%C9%94%C9%94_waa_(Empire_coloniaux)
1423
['Catégorie:!Main category', 'Catégorie:Abalaa sɔsaa', 'Catégorie:Itaalii', 'Catégorie:Keesi maɖa', 'Catégorie:Takayɩsɩ mayaa', 'Catégorie:Ɛyaa sɔsaa']
https://kbp.wikipedia.org/wiki/Dante_Alighieri
1424
['Catégorie:!Main category', 'Catégorie:Tɔɔnasɩ']
https://kbp.wikipedia.org/wiki/Aka%C5%8Bgaa
1425
['Catégorie:Fenasɩ']
https://kbp.wikipedia.org/wiki/%C6%89oma%C9%A3_fena%C9%A3
1426
['Catégorie:Pɩnzɩ', 'Catégorie:Pɩnzɩ mɩnɩŋ nɛɛlɛ']
https://kbp.wikipedia.org/wiki/1944
1427
['Catégorie:Ajɛya', 'Catégorie:Cɛɛkɩ', 'Catégorie:Eerɔpʋ']
https://kbp.wikipedia.org/wiki/C%C9%9B%C9%9Bk%C9%A9
1428
['Catégorie:Afrɩka', 'Catégorie:Togo', 'Catégorie:Togo taa tɛtʋ', 'Catégorie:Tɛtʋ sɔsɔtʋ']
https://kbp.wikipe

1484
['Catégorie:Pɩnzɩ', 'Catégorie:Pɩnzɩ mɩnɩŋ nɛɛlɛ nɛ kʋɖʋm']
https://kbp.wikipedia.org/wiki/2001
1485
['Catégorie:!Main category', 'Catégorie:Abalaa sɔsaa', 'Catégorie:Keɖeyaɣ laɖaa', 'Catégorie:Pɛliziki', 'Catégorie:Ɛyaa sɔsaa']
https://kbp.wikipedia.org/wiki/Pierre_Paul_Rubens
1486
['Catégorie:!Main category', 'Catégorie:Afrɩka', 'Catégorie:Ajɛya', 'Catégorie:Naajeeriya']
https://kbp.wikipedia.org/wiki/Naajeeriya
1487
['Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Lɩm tɔm']
https://kbp.wikipedia.org/wiki/L%C9%A9%C5%8Bgam%CA%8B%CA%8B
1488
['Catégorie:!Main category', 'Catégorie:Ñɩɣlɩm']
https://kbp.wikipedia.org/wiki/%C6%89%C9%94m
1489
['Catégorie:!Main category', 'Catégorie:Afrɩka', 'Catégorie:Ajɛya', 'Catégorie:Simbaabuwee']
https://kbp.wikipedia.org/wiki/Simbaabuwee
1490
['Catégorie:Afrɩka', 'Catégorie:Caanaʋ tɔm']
https://kbp.wikipedia.org/wiki/Afrika_k%C9%A9kp%C9%9Bd%CA%8A%CA%8A_t%C9%9B_t%C9%94m_(Histoire_de_l%27Afrique_subsaharienne)
1491
['Catégorie:!Main category']
https://kbp

1548
['Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Tɛtɛ wondu cikpetu', 'Catégorie:Wezuu lɛɣtʋ']
https://kbp.wikipedia.org/wiki/K%C9%9B%C9%9Bma%C9%A3
1549
['Catégorie:!Main category', 'Catégorie:Fiziiki']
https://kbp.wikipedia.org/wiki/Paraamiini
1550
['Catégorie:Kpɛli pɩyʋʋ pɩlʋʋ kpata']
https://kbp.wikipedia.org/wiki/L%C9%A9t%C9%A9y%C9%94m
1551
['Catégorie:!Main category', 'Catégorie:Fiziiki', 'Catégorie:Siimi']
https://kbp.wikipedia.org/wiki/Naz%C9%A9y%CA%8A_pe_(%C3%A9lectron)
1552
['Catégorie:Afrɩka', 'Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Politiki laɖaa', 'Catégorie:Tanzani']
https://kbp.wikipedia.org/wiki/Julius_Kambarage_Nyerere
1553
['Catégorie:Etaazuunii']
https://kbp.wikipedia.org/wiki/Arakanzaa
1554
['Catégorie:!Main category', 'Catégorie:Paɣtʋ lɛɣtʋ']
https://kbp.wikipedia.org/wiki/Pa%C9%A3t%CA%8A_(Loi)
1555
['Catégorie:!Main category', 'Catégorie:Wondu', 'Catégorie:Ñɩɣtʋ']
https://kbp.wikipedia.org/wiki/Z%C9%A9%C5%8Bk%C9%A9
1556
['Catégorie:Aŋgilitɛɛrɩ', 'Catégorie:Eerɔpʋ'

1612
['Catégorie:!Main category']
https://kbp.wikipedia.org/wiki/L%CA%8B%C5%8Bg%C9%A9y%C9%9B_he%C9%A3lim
1613
['Catégorie:Alɩwaatʋ', 'Catégorie:Kabɩyɛ takayɩhatʋ']
https://kbp.wikipedia.org/wiki/P%C9%A9na%C9%A3_h%C9%94%C9%94l%CA%8A%CA%8A
1614
['Catégorie:Pɩnzɩ', 'Catégorie:Pɩnzɩ mɩnɩŋ nɛɛlɛ']
https://kbp.wikipedia.org/wiki/1927
1615
['Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Maɣzʋʋ lɛɣtʋ']
https://kbp.wikipedia.org/wiki/%C6%90y%CA%8A_y%C9%94%C9%94da%C9%A3
1616
['Catégorie:!Main category', 'Catégorie:Minziiki', 'Catégorie:Minziiki wondu']
https://kbp.wikipedia.org/wiki/K%C9%94%C9%94r%C9%A9
1617
['Catégorie:!Main category']
https://kbp.wikipedia.org/wiki/Patrice_Lumumba
1618
['Catégorie:Kabɩyɛ takayɩhatʋ', 'Catégorie:Maɣzʋʋ lɛɣtʋ']
https://kbp.wikipedia.org/wiki/Ma%C9%A3z%C9%A9m_f%C9%9By%C9%A9_t%CA%8A
1619
['Catégorie:!Main category']
https://kbp.wikipedia.org/wiki/Kulukulu_kuduyuu_(Tour)
1620
['Catégorie:Siimi', 'Catégorie:Siimi taŋgayɩ ñɩɩɩ waa', 'Catégorie:Ñɩɣtʋ']
https://kbp.wikipedi

In [18]:
df2 = pd.DataFrame(list(zip(articlesList,contentList, urlList,categoryList)), 
               columns =['title','Content', 'Url', 'category']) 
len(df2)

1669

In [21]:
print(df2.head())

                     title                                            Content  \
0               Wɩsɩ kʋdɔŋ  Payaɣ wɩsɩ kʋdɔŋ ɖɔɖɔ se pʋtʋ kʋdɔŋ. Kʋdɔŋ piy...   
1      Nima yɔɔ lɩmaɣzɩoyɛ  Nima yɔɔ lɩmaɣzɩyɛ kɛ pɩlɩʋ ŋgʋ kɩ-taa pañɩnɩɣ...   
2                     Bɔrɩ  Bɔrɩ kɛ kpɛlɩ kpɛlɛkʊʊ tɛ pɩyʊ mbʊ ɛ-ñʊʊ mayaɣ...   
3                 Avicenna  Avicenne, yaa Ibn Sīnā1 (Persan kʊnʊŋ taa lɛ, ...   
4  Papuwazii Kinee Kɩfalʊʊ  Papuwazii Kinee Kɩfalʊʊ kɛna Oseyanii ilemiye ...   

                                                 Url  \
0  https://kbp.wikipedia.org/wiki/W%C9%A9s%C9%A9_...   
1  https://kbp.wikipedia.org/wiki/Nima_y%C9%94%C9...   
2      https://kbp.wikipedia.org/wiki/B%C9%94r%C9%A9   
3            https://kbp.wikipedia.org/wiki/Avicenna   
4  https://kbp.wikipedia.org/wiki/Papuwazii_Kinee...   

                                            category  
0      [Catégorie:!Main category, Catégorie:Kʋdʋmɩŋ]  
1   [Catégorie:Fiziiki, Catégorie:Kabɩyɛ takayɩhatʋ]  
2          

In [28]:
df2.to_csv("articlesByCategories.csv", index=False)

In [27]:
df2.head()

,title,Content,Url,category
0,Wɩsɩ kʋdɔŋ,Payaɣ wɩsɩ kʋdɔŋ ɖɔɖɔ se pʋtʋ kʋdɔŋ. Kʋdɔŋ piy...,https://kbp.wikipedia.org/wiki/W%C9%A9s%C9%A9_...,"[Catégorie:!Main category, Catégorie:Kʋdʋmɩŋ]"
1,Nima yɔɔ lɩmaɣzɩoyɛ,Nima yɔɔ lɩmaɣzɩyɛ kɛ pɩlɩʋ ŋgʋ kɩ-taa pañɩnɩɣ...,https://kbp.wikipedia.org/wiki/Nima_y%C9%94%C9...,"[Catégorie:Fiziiki, Catégorie:Kabɩyɛ takayɩhatʋ]"
2,Bɔrɩ,Bɔrɩ kɛ kpɛlɩ kpɛlɛkʊʊ tɛ pɩyʊ mbʊ ɛ-ñʊʊ mayaɣ...,https://kbp.wikipedia.org/wiki/B%C9%94r%C9%A9,[Catégorie:Kpɛli pɩyʋʋ pɩlʋʋ kpata]
3,Avicenna,"Avicenne, yaa Ibn Sīnā1 (Persan kʊnʊŋ taa lɛ, ...",https://kbp.wikipedia.org/wiki/Avicenna,[Catégorie:!Main category]
4,Papuwazii Kinee Kɩfalʊʊ,Papuwazii Kinee Kɩfalʊʊ kɛna Oseyanii ilemiye ...,https://kbp.wikipedia.org/wiki/Papuwazii_Kinee...,"[Catégorie:Aazii, Catégorie:Ajɛya, Catégorie:P..."
